<a href="https://colab.research.google.com/github/vs1991/Recommendation-System-/blob/main/Content_Based_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive

In [7]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Movie Recommendation System"


In [10]:
#changing the working directory
%cd /content/gdrive/My Drive/Movie Recommendation System
#Check the present working directory using pwd command

/content/gdrive/My Drive/Movie Recommendation System


In [11]:
ls

credits.csv   knowledge.csv    metadata_clean.csv   ratings_small.csv
kaggle.json   links.csv        movies_metadata.csv
keywords.csv  links_small.csv  ratings.csv


In [12]:
import pandas as pd
import numpy as np 

In [13]:
#we need to add plot feature into our cleaned model 
orig_df=pd.read_csv('movies_metadata.csv')
orig_df.head(3)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [14]:
# Function to convert all non-integer IDs to NaN
def clean_ids(x):
  try:
    return int(x)
  except:
      return np.nan
#Clean the ids of df
orig_df['id'] = orig_df['id'].apply(clean_ids)




In the previous recommendation system (Simple recommnender system ) the model doesn't take in consider the individual user's preference.The Knowledge base recommender did take account of user's preference for genres,timelines and duration,but the model remained generic.

Example:

<span style='color:Blue'><span style='background:yellow'>**Imagine Vishnu likes the movies Dark knight ,Iron man and Man of Steel.It can be said that these movies belong to superhero movies taste.However our last model would not be abel to capture this.Best it did was suggest all action movies,which is a superset of superhero movies.**  </span>

---

It is also possible that 2 movies have the same genre ,timeline and duration,but differ hugely in there audience .

Example 

<span style='color:Blue'><span style='background:yellow'> **the hangover and forgetting sarah marshall were released in the 21st century,both lasted 2 hrs and both were comedies .However the audience who enjoyed these movies were different.**
    
    
**One fix to this problem** is to ask the user for more metadata.For instance if we add a sub-genre input,the user would be able to input values such as superhero,black comedy,and romantic comedy and we can obtain good results.But there are some **side effects** to this
    
1.) We do not posses data of sub genre and even if we did the user are unlikely to posses this knowledge.
    
2.) Even if they did user wont have the patience to include this data 

# Content Based recommender system 
 We will build  2 types of content based recommender system 
 
 1.) <span style='background:yellow'>**Plot-description based recommender :** 
    
    This model takes the description and taglines of most movies and provides recommendation based on similar plots
    
 2.) <span style='background:yellow'>**MetaData-based-Recommender:**
    
    This model takes into account host of features(genre,keywords,cast and crew) and provides recommendations that are the most similart to mentioned features

In [15]:
#lets read the cleaned saved dataset we made in knowledge based recommender system 
import pandas as pd 
import pandas as np 

In [16]:
#we need to add plot feature into our cleaned model 
df=pd.read_csv('metadata_clean.csv')
df.shape

(45466, 6)

# Plot based recommender system

Our plot description-based recommender will take in a movie title as an argument and
output a list of movies that are most similar based on their plots. These are the steps we are
going to perform in building this model:

1. Obtain the data required to build the model

2. Create TF-IDF vectors for the plot description (or overview) of every movie

3. Compute the pairwise cosine similarity score of every movie

4. Write the recommender function that takes in a movie title as an argument and
outputs movies most similar to it based on the plot

In [17]:
#adding the useful feature into cleaned dataset 

df['overview'],df['id']=orig_df['overview'],orig_df['id']
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862.0
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844.0
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602.0
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357.0
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862.0


In [18]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english')

In [19]:
#replacing nan values with eimpty string 
df['overview']=df['overview'].fillna('')

tfidf_matrix=tfidf.fit_transform(df['overview'][0:20000])

In [20]:
tfidf_matrix.shape

(20000, 47487)

In [21]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [22]:
indices=pd.Series(df.index,index=df['title']).drop_duplicates()
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45461
Century of Birthing            45462
Betrayal                       45463
Satan Triumphant               45464
Queerama                       45465
Length: 45466, dtype: int64

In [23]:
# Function that takes in movie title as input and gives recommendations
def content_recommender(title, cosine_sim=cosine_sim, df=df,indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:11]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar movies
    
    return df['title'].iloc[movie_indices]

In [24]:
content_recommender('Jumanji')

6166                       Brainscan
8801                         Quintet
17223                 The Dark Angel
9503                       Word Wars
13601    The Mindscape of Alan Moore
16843                         DeVour
8079                         Masques
6055                Poolhall Junkies
19726                 Wreck-It Ralph
2486                        eXistenZ
Name: title, dtype: object

# MetaData Based Recommender

In [25]:
cred_df=pd.read_csv('credits.csv')
cred_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [26]:
key_df=pd.read_csv('keywords.csv')
key_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [27]:
#Filter all rows that have a null ID
df = df[df['id'].notnull()]
# Merge keywords and credits into your main metadata dataframe
df = df.merge(cred_df, on='id')
df = df.merge(key_df, on='id')
#Display the head of the merged df
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [28]:
df.iloc[0]['keywords']

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [29]:
df.iloc[0]['cast']

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [30]:
df.iloc[0]['crew']

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [31]:
#converting stringified objects into python dictionary

from ast import literal_eval

features=['cast','crew','keywords','genres']

for feature in features:
  df[feature]=df[feature].apply(literal_eval)

In [32]:
df.iloc[0]['crew'][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [37]:
# Extract the director's name. If director is not listed, return NaN
def get_director(x):
  for crew_member in x:
    if crew_member['job'] == 'Director':
      return crew_member['name']
  return ''

In [38]:
df['director']=df['crew'].apply(get_director)
df['director'].head(5)

0      John Lasseter
1       Joe Johnston
2      Howard Deutch
3    Forest Whitaker
4      Charles Shyer
Name: director, dtype: object

In [39]:
df.iloc[0]['keywords'][0]

{'id': 931, 'name': 'jealousy'}

In [40]:
def generate_list(x):
  if isinstance(x,list):
    names=[ele['name'] for ele in x]
    if len(names)>3:
       names=names[:3]
    return names 

  return[]



In [41]:
#Apply the generate_list function to cast and keywords
df['cast'] = df['cast'].apply(generate_list)


In [42]:
df['keywords'] = df['keywords'].apply(generate_list)


In [43]:
#Only consider a maximum of 3 genres
df['genres'] = df['genres'].apply(lambda x: x[:3])

In [44]:
# Print the new features of the first 5 movies along with title
df[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


we are going to use a vectorizer to build document vectors. If two
actors had the same first name (say, Ryan Reynolds and Ryan Gosling), the vectorizer will
treat both Ryans as the same, although they are clearly different entities. This will impact
the quality of the recommendations we receive. If a person likes Ryan Reynolds' movies, it
doesn't imply that they like movies by all Ryans. 

In [45]:
# Function to sanitize data to prevent ambiguity.
# Removes spaces and converts to lowercase
def sanitize(x):
  if isinstance(x, list):
    #Strip spaces and convert to lowercase
    return [str.lower(i.replace(" ", "")) for i in x]
  else:
    #Check if director exists. If not, return empty string
    if isinstance(x, str):
      return str.lower(x.replace(" ", ""))
    else:
       return ''



In [46]:
#Apply the generate_list function to cast, keywords, director and genres
for feature in ['cast', 'director', 'genres', 'keywords']:
  df[feature] = df[feature].apply(sanitize)

In [47]:
df[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[tomhanks, timallen, donrickles]",johnlasseter,"[jealousy, toy, boy]","[animation, comedy, family]"
1,Jumanji,"[robinwilliams, jonathanhyde, kirstendunst]",joejohnston,"[boardgame, disappearance, basedonchildren'sbook]","[adventure, fantasy, family]"
2,Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret]",howarddeutch,"[fishing, bestfriend, duringcreditsstinger]","[romance, comedy]"


In the plot description-based recommender, we worked with a single overview feature,
which was a body of text. Therefore, we were able to apply our vectorizer directly.
However, this is not the case with our metadata-based recommender. We have four
features to work with, of which three are lists and one is a string. What we need to do is
create a soup that contains the actors, director, keywords, and genres. This way, we can
feed this soup into our vectorizer and perform similar follow-up steps to before:

In [48]:
#Function that creates a soup out of the desired metadata
def create_soup(x):
  return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [49]:
# Create the new soup feature
df['soup'] = df.apply(create_soup, axis=1)


In [50]:
df[['title', 'cast', 'director', 'keywords', 'genres','soup']].head(3)

,title,cast,director,keywords,genres,soup
0,Toy Story,"[tomhanks, timallen, donrickles]",johnlasseter,"[jealousy, toy, boy]","[animation, comedy, family]",jealousy toy boy tomhanks timallen donrickles ...
1,Jumanji,"[robinwilliams, jonathanhyde, kirstendunst]",joejohnston,"[boardgame, disappearance, basedonchildren'sbook]","[adventure, fantasy, family]",boardgame disappearance basedonchildren'sbook ...
2,Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret]",howarddeutch,"[fishing, bestfriend, duringcreditsstinger]","[romance, comedy]",fishing bestfriend duringcreditsstinger walter...


Instead of using TF-IDFVectorizer, we will be using CountVectorizer. This is because using
TF-IDFVectorizer will accord less weight to actors and directors who have acted and
directed in a relatively larger number of movies.

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
#Define a new CountVectorizer object and create vectors for the soup
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'][0:10000])


In [52]:
count_matrix.shape

(10000, 20264)

In [53]:
#Import cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity
#Compute the cosine similarity score (equivalent to dot product for tf-idfvectors)
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [58]:
# Reset index of your df and construct reverse mapping again
#df = df.reset_index()
indices2 = pd.Series(df.index, index=df['title'])

In [59]:
content_recommender('The Lion King',cosine_sim2,df,indices2)

3336                Creature Comforts
3497                     Time Masters
3724    Thomas and the Magic Railroad
7090                    Teacher's Pet
1018              So Dear to My Heart
2787                       Thumbelina
4949            The Flight of Dragons
1648                 Ill Gotten Gains
3487       Jails, Hospitals & Hip-Hop
651         James and the Giant Peach
Name: title, dtype: object